In [1]:
import csv
import pandas as pd
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
import numpy as np

In [2]:
data=pd.read_csv('cleaned_2.csv')
data

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDB14,20.25,Regular,0.171939,Canned,92.5120,OUT010,1998.0,Small,Tier 3,Grocery Store,186.4240
1,FDS52,8.89,Low Fat,0.009163,Frozen Foods,101.7016,OUT010,1998.0,Small,Tier 3,Grocery Store,101.2016
2,NCN07,18.50,Low Fat,0.056816,Others,132.1284,OUT010,1998.0,Small,Tier 3,Grocery Store,263.6568
3,FDI32,17.70,Low Fat,0.291865,Fruits and Vegetables,115.1834,OUT010,1998.0,Small,Tier 3,Grocery Store,345.5502
4,FDR47,17.85,Low Fat,0.000000,Breads,196.5794,OUT010,1998.0,Small,Tier 3,Grocery Store,585.2382
...,...,...,...,...,...,...,...,...,...,...,...,...
8324,NCU06,17.60,Low Fat,0.074523,Household,228.8010,OUT049,1999.0,Medium,Tier 1,NaN,3215.8140
8325,FDH35,18.25,Low Fat,0.060342,Starchy Foods,165.9526,OUT049,1999.0,Medium,Tier 1,NaN,1644.5260
8326,NCY42,6.38,Low Fat,0.015186,Household,144.9470,OUT049,1999.0,Medium,Tier 1,NaN,3435.5280
8327,DRL49,13.15,Low Fat,0.056517,Soft Drinks,143.4812,OUT049,1999.0,Medium,Tier 1,NaN,854.8872


In [3]:
data = data[data['Item_Outlet_Sales'].notnull()]


In [4]:
item_avgweight=data.pivot_table(values='Item_Weight', index='Item_Identifier')
print(item_avgweight)

                 Item_Weight
Item_Identifier             
DRA12                 11.600
DRA24                 19.350
DRA59                  8.270
DRB01                  7.390
DRB13                  6.115
...                      ...
NCZ30                  6.590
NCZ41                 19.850
NCZ42                 10.500
NCZ53                  9.600
NCZ54                 14.650

[1555 rows x 1 columns]


In [5]:
data[:][data['Item_Identifier']=='DRI11']

def input_weight(cols):
    weight=cols[0]
    identifier=cols[1]
    if(pd.isnull(weight)):
        return item_avgweight['Item_Weight'][item_avgweight.index==identifier].get(0)
    else:
        return weight

print('Og missing: %d'%sum(data['Item_Weight'].isnull()))
data['Item_Weight']=data[['Item_Weight', 'Item_Identifier']].apply(input_weight,axis=1)
print('Final missing: %d'%sum(data['Item_Weight'].isnull()))

print(list(data['Item_Weight'].unique()))

Og missing: 4
Final missing: 4
[20.25, 8.89, 18.5, 17.7, 17.85, 11.8, 15.5, 12.85, 14.65, 6.67, 13.65, 8.93, 6.4, 7.905, 18.2, 19.0, 9.8, 15.2, 10.5, 7.075, 16.35, 21.35, 14.15, 10.895, 20.75, 18.1, 15.6, 8.71, 19.7, 12.15, 8.975, 8.785, 17.25, 13.8, 12.1, 17.5, 7.825, 7.06, 19.35, 20.2, 11.395, 20.85, 9.5, 15.35, 10.3, 17.75, 4.61, 7.68, 13.6, 12.5, 6.635, 15.7, 16.75, 16.5, 18.35, 15.25, 5.03, 6.59, 13.5, 20.0, 16.6, 4.88, 16.1, 9.0, 20.5, 12.8, 10.195, 15.3, 5.785, 18.25, 9.6, 6.61, 19.6, 15.85, 8.115, 5.175, 6.035, 12.6, 7.3, 9.195, 8.315, 19.75, 14.1, 12.35, 20.7, 17.1, 11.5, 8.88, 19.1, 14.5, 7.72, 7.575, 12.65, 17.6, 19.5, 7.285, 11.6, 7.655, 21.0, 12.3, 9.3, 16.7, 19.2, 14.35, 6.425, 5.325, 7.605, 8.02, 13.0, 6.825, 11.85, 6.115, 14.85, 20.35, 9.27, 6.26, 8.6, 16.25, 13.15, 5.365, 11.1, 5.73, 7.5, 18.85, 7.67, 10.695, 10.8, 20.6, 6.3, 6.155, 13.3, 14.0, 9.17, 21.2, 7.05, 6.44, 8.52, 6.715, 13.35, 5.825, 20.1, 7.435, 19.25, 5.425, 10.85, 14.8, 11.35, 6.86, 6.8, 5.15, 6.32, 7.725

In [6]:
from scipy.stats import mode
outlet_sizemode=data.pivot_table(values='Outlet_Size', columns='Outlet_Type', aggfunc=lambda x:x.mode())
outlet_sizemode

Outlet_Type,Grocery Store,Supermarket Type1,Supermarket Type2,Supermarket Type3
Outlet_Size,Small,Small,Medium,Medium


In [7]:
def input_size(cols):
    size=cols[0]
    outlet_type=cols[1]
    if(pd.isnull(size)):
        return outlet_sizemode.loc['Outlet_Size'][outlet_sizemode.columns==outlet_type].get(0)
    else:
        return size

print('Og missing: %d'%sum(data['Outlet_Size'].isnull()))
data['Outlet_Size']=data[['Outlet_Size', 'Outlet_Type']].apply(input_size,axis=1)
print('Final missing: %d'%sum(data['Outlet_Size'].isnull()))

print(list(data['Outlet_Size'].unique()))

Og missing: 0
Final missing: 0
['Small', 'High', 'Medium']


In [8]:
outlet_location=data.pivot_table(values='Outlet_Location_Type', columns='Outlet_Type', aggfunc=lambda x:x.mode())
outlet_location

Outlet_Type,Grocery Store,Supermarket Type1,Supermarket Type2,Supermarket Type3
Outlet_Location_Type,Tier 3,Tier 2,Tier 3,Tier 3


In [9]:
def input_location(cols):
    loc=cols[0]
    outlet_type=cols[1]
    if(pd.isnull(loc)):
        return outlet_location.loc['Outlet_Location_Type'][outlet_location.columns==outlet_type].get(0)
    else:
        return loc

print('Og missing: %d'%sum(data['Outlet_Location_Type'].isnull()))
data['Outlet_Location_Type']=data[['Outlet_Location_Type', 'Outlet_Type']].apply(input_location,axis=1)
print('Final missing: %d'%sum(data['Outlet_Location_Type'].isnull()))

print(list(data['Outlet_Location_Type'].unique()))

Og missing: 0
Final missing: 0
['Tier 3', 'Tier 2', 'Tier 1']


In [10]:
item_avgvisibility=data.pivot_table(values='Item_Visibility', index='Item_Identifier')
print(item_avgvisibility)

                 Item_Visibility
Item_Identifier                 
DRA12                   0.031956
DRA24                   0.048062
DRA59                   0.134718
DRB01                   0.082126
DRB13                   0.008002
...                          ...
NCZ30                   0.024956
NCZ41                   0.051623
NCZ42                   0.009044
NCZ53                   0.028575
NCZ54                   0.080426

[1557 rows x 1 columns]


In [11]:
def input_visibility(cols):
    visibility=cols[0]
    identifier=cols[1]
    if(pd.isnull(visibility) or visibility==0):
        return item_avgvisibility['Item_Visibility'][item_avgvisibility.index==identifier].get(0)
    else:
        return visibility

print('Og missing: %d'%sum(data['Item_Visibility'].isnull()))
data['Item_Visibility']=data[['Item_Visibility', 'Item_Identifier']].apply(input_visibility,axis=1)
print('Final missing: %d'%sum(data['Item_Visibility'].isnull()))

print(list(data['Item_Visibility'].unique()))

Og missing: 2
Final missing: 2
[0.171938781, 0.009163216, 0.056816465, 0.291865402, 0.07008762299999999, 0.098312421, 0.042464962, 0.165694219, 0.080426149, 0.150554711, 0.13497562800000001, 0.049754975, 0.148392623, 0.016758568999999997, 0.082551042, 0.103481781, 0.151420934, 0.031860325, 0.0090443174, 0.03777249, 0.054463442, 0.11519471699999999, 0.05837308, 0.228993134, 0.14940405699999998, 0.29788371199999997, 0.022059594, 0.11168586800000001, 0.23303981699999998, 0.089083914, 0.030815427000000003, 0.01516265, 0.06093978666666666, 0.020539695166666667, 0.079113947, 0.162462044, 0.028164527, 0.028009382000000003, 0.133281968, 0.073480266, 0.137697119, 0.146973462, 0.11035679699999999, 0.03616416, 0.062606583, 0.03668952862500001, 0.019314961000000002, 0.16135017599999998, 0.09846497900000001, 0.021343732, 0.112985849, 0.20529482699999999, 0.011314423, 0.10599465400000001, 0.118672537, 0.050111641, 0.063430788, 0.016726506000000002, 0.12341737, 0.07714555299999999, 0.075661985, 0.126

In [12]:
years = data.pivot_table(values='Outlet_Establishment_Year', index='Outlet_Identifier')
def fill_years(c):
    year = c[0]
    idval = c[1]
    if pd.isnull(year) == True:
        return years['Outlet_Establishment_Year'][years.index == idval].get(0)
    else:
        return year
    
print('Og missing: %d'%sum(data['Outlet_Establishment_Year'].isnull()))
data['Outlet_Establishment_Year'] = data[['Outlet_Establishment_Year', 'Outlet_Identifier']].apply(fill_years, axis=1)
print('Og missing: %d'%sum(data['Outlet_Establishment_Year'].isnull()))

print(list(data['Outlet_Establishment_Year'].unique()))


# data.isnull().sum()/data.shape[0]*100

Og missing: 2
Og missing: 2
[1998.0, 1985.0, 1987.0, 2009.0, 2002.0, 2007.0, 2004.0, 1997.0, nan, 1999.0]


In [13]:
MRP = data.pivot_table(values='Item_MRP', index='Item_Identifier')
def fill_MRP(c):
    mrp = c[0]
    idval = c[1]
    if pd.isnull(mrp) == True:
        return MRP['Item_MRP'][MRP.index == idval].get(0)
    else:
        return mrp
    
print('Og missing: %d'%sum(data['Item_MRP'].isnull()))
data['Item_MRP'] = data[['Item_MRP', 'Item_Identifier']].apply(fill_MRP, axis=1)
print('Og missing: %d'%sum(data['Item_MRP'].isnull()))
print(list(data['Item_MRP'].unique()))

# data.isnull().sum()/data.shape[0]*100

Og missing: 0
Og missing: 0
[92.512, 101.7016, 132.1284, 115.1834, 196.5794, 81.4618, 81.5934, 39.7164, 161.5552, 130.0626, 260.0936, 152.134, 41.5796, 250.4408, 147.3734, 228.27200000000002, 194.3478, 235.1248, 238.3248, 98.0068, 163.8842, 258.3278, 243.9828, 263.4568, 192.8478, 156.7288, 56.0614, 184.7924, 49.5376, 54.593, 252.0698, 101.899, 46.5402, 123.0414, 99.2068, 55.893, 178.166, 141.718, 63.2826, 58.8904, 47.1034, 184.1292, 62.051, 150.1708, 192.2478, 32.89, 164.6526, 196.6768, 188.653, 58.3588, 185.424, 172.8396, 122.973, 150.5682, 192.2162, 165.9816, 65.4826, 176.037, 86.0198, 39.8506, 181.36599999999999, 35.7532, 178.7318, 191.9504, 211.0244, 85.3198, 122.0756, 121.4098, 81.096, 169.6132, 173.9422, 52.2298, 34.9558, 169.4816, 38.319, 90.883, 224.8404, 119.1808, 221.8456, 75.0696, 104.0332, 119.244, 89.8856, 84.4224, 95.5094, 226.2062, 166.9184, 188.4898, 168.8816, 218.3166, 93.6462, 155.2972, 106.7622, 155.5998, 253.0356, 161.0578, 146.3076, 101.0016, 143.6444, 179.766, 198

In [14]:
Fat = data.pivot_table(values='Item_Fat_Content', index='Item_Identifier', aggfunc=lambda x:x.mode())


def fill_fat(c):
    fat = c[0]
    idval = c[1]
    if pd.isnull(fat) == True:
#         print(fat)
        if len(Fat['Item_Fat_Content'][Fat.index == idval]):
#             if "[" in Fat['Item_Fat_Content'][Fat.index == idval][0]:
#                 return 'Low Fat'
#             else:
            return Fat['Item_Fat_Content'][Fat.index == idval][0] 
        else:
            return Fat['Item_Fat_Content'][Fat.index == idval]
    elif fat == 'LF' or fat == 'low fat':
#         print('here')
        return 'Low Fat'
    elif fat == 'reg':
#         print('reg')
        return 'Regular'
    else:
        return fat
    
print('Og missing: %d'%sum(data['Item_Fat_Content'].isnull()))
data['Item_Fat_Content'] = data[['Item_Fat_Content', 'Item_Identifier']].apply(fill_fat, axis=1)
print('Og missing: %d'%sum(data['Item_Fat_Content'].isnull()))

print(list(data['Item_Fat_Content'].unique()))
# data.isnull().sum()/data.shape[0]*100

Og missing: 0
Og missing: 0
['Regular', 'Low Fat']


In [15]:
data.isnull().sum()/data.shape[0]*100

Item_Identifier              0.000000
Item_Weight                  0.048025
Item_Fat_Content             0.000000
Item_Visibility              0.024012
Item_Type                    0.900468
Item_MRP                     0.000000
Outlet_Identifier            1.008524
Outlet_Establishment_Year    0.024012
Outlet_Size                  0.000000
Outlet_Location_Type         0.000000
Outlet_Type                  0.924481
Item_Outlet_Sales            0.000000
dtype: float64

In [16]:
# data.to_csv("cleaned.csv",index=False)
data[data['Outlet_Size'].isnull() == True]

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales


In [17]:
data[data['Outlet_Identifier'] == 'OUT013']

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
552,FDG16,15.250,Low Fat,0.089742,Frozen Foods,213.7192,OUT013,1987.0,Small,Tier 3,Supermarket Type1,1078.5960
553,FDZ36,6.035,Regular,0.065729,Baking Goods,188.3240,OUT013,1987.0,Small,Tier 3,Supermarket Type1,1491.3920
554,FDO31,6.760,Regular,0.028959,Fruits and Vegetables,78.3960,OUT013,1987.0,Small,Tier 3,Supermarket Type1,1757.7120
555,FDK56,9.695,Low Fat,0.129890,Fruits and Vegetables,185.3898,OUT013,1987.0,Small,Tier 3,Supermarket Type1,3554.7062
556,FDM51,11.800,Regular,0.025905,Meat,102.4674,OUT013,1987.0,Small,Tier 3,Supermarket Type1,2037.3480
...,...,...,...,...,...,...,...,...,...,...,...,...
5994,FDG60,20.350,Low Fat,0.060649,Baking Goods,232.7616,OUT013,1987.0,High,Tier 2,Supermarket Type1,2812.3392
5995,FDU20,19.350,Regular,0.021440,Fruits and Vegetables,119.7098,OUT013,1987.0,High,Tier 2,Supermarket Type1,1325.6078
5996,FDT31,19.750,Low Fat,0.012438,Fruits and Vegetables,189.9872,OUT013,1987.0,High,Tier 2,Supermarket Type1,3781.7440
5997,NCK30,14.850,Low Fat,0.060928,Household,254.8698,OUT013,1987.0,High,Tier 2,Supermarket Type1,4819.7262


In [18]:
data[data['Item_Visibility'].isnull()]

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
1744,FDC23,18.0,Low Fat,NaN,Starchy Foods,178.3686,OUT018,2009.0,Medium,Tier 3,Supermarket Type2,3199.8348
6290,FDQ60,NaN,Regular,NaN,Baking Goods,121.2098,OUT019,1985.0,Small,Tier 1,Grocery Store,120.5098


In [19]:
data[data['Item_Identifier'] == 'FDC23']

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
1744,FDC23,18.0,Low Fat,NaN,Starchy Foods,178.3686,OUT018,2009.0,Medium,Tier 3,Supermarket Type2,3199.8348


In [20]:
data[data['Item_Identifier'] == 'FDQ60']

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
6290,FDQ60,NaN,Regular,NaN,Baking Goods,121.2098,OUT019,1985.0,Small,Tier 1,Grocery Store,120.5098


In [21]:
data[data['Outlet_Establishment_Year'].isnull()]

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
4502,NCK53,11.6,Low Fat,0.037794,Health and Hygiene,99.3042,NaN,NaN,Small,Tier 2,Supermarket Type1,2281.6966
4860,FDE40,15.6,Regular,0.099345,Dairy,63.6194,NaN,NaN,Small,Tier 2,Supermarket Type1,743.0328


In [22]:
y = data.pivot_table(values=['Outlet_Location_Type','Outlet_Establishment_Year','Outlet_Type','Outlet_Size'], index='Outlet_Identifier', aggfunc=lambda x:x.mode())
y

,Outlet_Establishment_Year,Outlet_Location_Type,Outlet_Size,Outlet_Type
Outlet_Identifier,,,,
OUT010,1998.0,Tier 3,Small,Grocery Store
OUT013,1987.0,Tier 3,High,Supermarket Type1
OUT017,2007.0,Tier 2,Small,Supermarket Type1
OUT018,2009.0,Tier 3,Medium,Supermarket Type2
OUT019,1985.0,Tier 1,Small,Grocery Store
OUT027,1985.0,Tier 3,Medium,Supermarket Type3
OUT035,2004.0,Tier 2,Small,Supermarket Type1
OUT045,2002.0,Tier 2,Small,Supermarket Type1
OUT046,1997.0,Tier 1,Small,Supermarket Type1


In [23]:
# data.to_csv("cleaned_3.csv",index=False)